# VK API

## Введение

Мы уже [учились выкачивать Интернет](https://github.com/elmiram/2016learnpython/blob/master/1%20%D0%A1%D0%B5%D0%BC%D0%B8%D0%BD%D0%B0%D1%80%20-%20urllib.ipynb). Однако этот способ добычи текстов (шахтёрская терминология тут неслучайна, по-английски и интеллектуальный анализ текста, и процесс нахождения данных для этого анализа называется *mining*) подходит не для всех сайтов. Некоторые ресурсы либо прямо запрещают автоматическое обращение к своему контенту, потому что это для них невыгодно (зарабатывают-то они на рекламе, которую показывают **людям**, а с роботов отдача нулевая) и излишне загружает сервера, либо просто организовывают свой сайт так, чтобы сама по себе загрузка страницы клиентом не давала ничего существенного, а всё нужное подгружалось уже потом с помощью программ на языке JavaScript. Запускать такие программы из питона несколько сложнее, так что всё это создаёт для компьютерного лингвиста дополнительные трудности. К таким сайтам относятся, например, и социальные сети: VK, Facebook, Instagram и пр.

Однако многие крупные ресурсы либо по доброте душевной, либо потому что это даёт и им кое-какую выгоду, встраивают в свою систему API: application programming interface, то есть средство для автоматизированного обращения к приложению (сайту). Через такую систему можно решать иногда довольно широкий спектр задач. Может быть, всё, что таким образом можно сделать, нам не нужно. Но вот получать тексты было бы полезно: в тех же социальных сетях люди пишут, во-первых, много (а для компьютерного лингвиста чем больше данных, тем лучше), во-вторых, на таком варианте языка, который приближен к разговорному (другие способы намайнить себе текстов такого рода гораздо затратнее). Попробуем познакомиться с инструментарием API на примере vk.com



## Как выглядит обращение к VK API

В принципе, VK API, как следует из [документации](https://vk.com/dev/openapi) придуман не для выкачивания текстов, а для создания веб-приложений на сторонних сайтах, которые бы могли взаимодействовать с vk.com. Но это всё равно не мешает нам воспользоваться такой возможностью в своих целях.

Практически всё выглядит следующим образом. На сайте vk.com есть специальные страницы, которые не предназначены для того, чтобы их открывать браузером, они ожидают именно автоматического обращения. Что значит *автоматическое обращение*? Это то самое, что мы проходили, когда учились выкачивать Интернет: программа на питоне (но в теории может быть и не на питоне) посылает серверу запрос, договаривается с ним и получает ответ. Вспомним, что для произвольной страницы это выглядит так:

In [1]:
import urllib.request  # импортируем модуль 
req = urllib.request.Request('https://habrahabr.ru/') # посылаем запрос
with urllib.request.urlopen(req) as response: # открываем соединение с сайтом
   html = response.read().decode('utf-8') # "читаем" ответ сервера (сайта) в переменную html

Для взаимодействия с vk.com нам потребуется тот же самый модуль urllib.request, а страницы, к которым мы будем обращаться, описаны в [документации VK API](https://vk.com/dev/openapi).

Но есть одна хитрость. Она в том, что страницы, к которым мы будем обращаться, ожидают не просто обращения, а передачи определённых параметров. Это логично: мы же должны сказать системе, что мы точно хотим сделать. Например, мы хотим получить какое-то количество записей со стены определённого пользователя. Тогда нужно сообщить, какой это пользователь. Сайт vk.com много чего может, но не читать наши мысли.

### Передача параметров

Как передать эти параметры сайту? Для этого тоже есть стандартные средства, но уже не придуманные разработчиками сайта, а прописанные в [протоколе HTTP](https://www.tutorialspoint.com/http/http_parameters.htm) давным-давно. Выглядит это так. В строке адреса, например, в браузере мы сначала пишем имя протокола (http или https), которое отделяется от всего, что идёт дальше последовательностью "://", потом пишется доменное имя (по сути, основной адрес сайта), к которому мы хотим обратиться (например, vk.com, после точки следует т.н. *доменная зона*), после косой черты далее следует адрес собственно страницы на сайте, к которой мы хотим обратиться: https://api.vk.com/method/wall.get А вот после адреса страницы мы можем в той же адресной строке передать уже собственно параметры. Место, где передаются параметры, отделяется от адреса страницы с помощью знака вопроса, а сами выглядят как пары ключ-значение, где ключ отделяется от значения с помощью знака равно: https://api.vk.com/method/wall.get?owner_id=1 Здесь есть параметр owner_id, который указывает на пользователя, стену которого мы хотим скачать, и значение этого параметра 1, то есть речь идёт об основателе соцсети и первом её пользователе Павле Дурове, страница которого открывается по адресу [https://vk.com/id1](https://vk.com/id1). Такой номер, то есть уникальный идентификатор (*id*) есть у каждого пользователя и у каждого сообщества. 

Если есть необходимость передать сразу несколько параметров, то они должны отделяться друг от друга знаком "аперсанд", то есть **&**: https://api.vk.com/method/wall.get?owner_id=1&count=10. Здесь появился второй параметр count, который говорит, что мы хотим скачать именно 10 записей со стены пользователя.

Так же выглядит и строка адреса при поисковом запросе, например, в Яндексе: https://yandex.ru/search/?text=соцсети


### Пробуем!

Итак, поехали, давайте скачаем две записи со стены Дурова. Чтобы скачивать записи со стен, у нас есть специальный метод VK API, он называется wall.get и выглядит как специальная страница на сайте vk.com: https://api.vk.com/method/wall.get Как работает этот метод, рассказано на его странице в документации: https://vk.com/dev/wall.get Среди полезного там есть список обязательных параметров, которые нужно передать методу, чтобы он сработал.
                
Часть адреса https://api.vk.com/method/ можно запомнить. Все остальные методы просто добавляются к нему: https://api.vk.com/method/wall.getComments, https://api.vk.com/method/wall.getById и т.д.            

In [2]:
import urllib.request  # импортируем модуль 
req = urllib.request.Request('https://api.vk.com/method/wall.get?owner_id=1&count=2') 
response = urllib.request.urlopen(req) # да, так тоже можно, не обязательно делать это с with, как в примере выше
result = response.read().decode('utf-8')

Посмотрим, что получилось:

In [3]:
print (result)

{"response":[234,{"id":1674764,"from_id":1,"to_id":1,"date":1491046870,"post_type":"post","text":"К 1 апреля в Трижды Краснознаменный завезли новые стикеры.","attachment":{"type":"photo","photo":{"pid":456263811,"aid":-7,"owner_id":1,"src":"https:\/\/pp.userapi.com\/c836333\/v836333001\/30a2f\/6e-bqVFCKiw.jpg","src_big":"https:\/\/pp.userapi.com\/c836333\/v836333001\/30a30\/Grb21yg2mfA.jpg","src_small":"https:\/\/pp.userapi.com\/c836333\/v836333001\/30a2e\/027Nqr_T3iw.jpg","src_xbig":"https:\/\/pp.userapi.com\/c836333\/v836333001\/30a31\/s2pr5YnsQ-w.jpg","src_xxbig":"https:\/\/pp.userapi.com\/c836333\/v836333001\/30a32\/vqXD6TjwUrc.jpg","src_xxxbig":"https:\/\/pp.userapi.com\/c836333\/v836333001\/30a33\/HO2a_wZlzd4.jpg","width":1200,"height":1200,"text":"","created":1491046798,"post_id":1674764,"access_key":"6b81100d8d5e38e742"}},"attachments":[{"type":"photo","photo":{"pid":456263811,"aid":-7,"owner_id":1,"src":"https:\/\/pp.userapi.com\/c836333\/v836333001\/30a2f\/6e-bqVFCKiw.jpg","s

Если вглядеться, то выйдет, что это просто питоновский словарь с ключом "response" и значением в виде массива с двумя элементами. Но так ли это?

In [4]:
type(result)

str

Нет, оказывается, это просто строка :( Неужели её ешщё нужно парсить? Вспоминать регулярные выражения? Нет! Вспомним лучше [формат json и модуль для работы с ним](https://github.com/elmiram/2016learnpython/blob/master/5%20%D0%A1%D0%B5%D0%BC%D0%B8%D0%BD%D0%B0%D1%80%20-%20json.ipynb).

In [5]:
import json
data = json.loads(result) 
print(type(data))

<class 'dict'>


Теперь, кажется, всё в порядке. Собственно, на том же занятии про json мы уже работали с api, только другого сайта: API GitHub.


## Извлечение информации

Мы получили какой-то ответ от API и даже превратили его в структуру данных, с которой удобно работать в питоне. Но что в этой структуре лежит? Обычно это json, в котором всё устроено как вложенные друг в друга словари и массивы, где ключи -- это какое-то общепонятное слово, а значение -- собственно информация. Подробно это описано в документации к API, но часто всё понятно и без дополнительных объяснений. Например, вот тут:

{

&nbsp;&nbsp;&nbsp;&nbsp;"post_type": "post",

&nbsp;&nbsp;&nbsp;&nbsp;"text": "К 1 апреля в Трижды Краснознаменный завезли новые стикеры.",

&nbsp;&nbsp;&nbsp;&nbsp;"attachment":

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;{

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"type":"photo"

...

ключ "post_type" говорит, что это за тип записи, а ключ "attachment" содержит информацию о том, что к этому посту приложено. Оказывается, фото (точнее, просто какая-то картинка).



Попробуем это извлечь.

In [7]:
print(data["response"][1]["text"])

К 1 апреля в Трижды Краснознаменный завезли новые стикеры.


Получилось! 

А если второй пост?

In [8]:
print(data["response"][2]["text"])

Если кто ещё не использует телеграм, то вот вам немножко доводов http://telegra.ph/Vozmozhnosti-Telegram-03-30<br><br>АПД: Звонки уже запустили


Снова получилось!

##  Задания

### 1. Смотрим на другие параметры метода wall.get

Что ещё можно передать методу wall.get, кроме id пользователя и числа постов? Как это применить?

### 2. Скачиваем комментарии

Комментарии к постам скачиваются с помощью другого метода: https://vk.com/dev/wall.getComments

Ему нужно передавать идентификаторы записи, комментарии к которой мы хотим получить (эти идентификаторы нам поставляет метод wall.get). Обратите внимание, что VK API позволяет за одно обращение скачать не больше 100 записей и 100 комментариев. Дурова комментируют много, так что всё сразу достать не получится. Но если немного подумать, то можно сделать и это. Как?

Если ваша собственная стена открыта и записи там доступны без авторизации, можете скачать её, и посчитать, кто вас больше комментирует. Какие у комментаторов самые частотные слова?